### Importing libraries

In [12]:
from NiaPy.algorithms.basic import FireflyAlgorithm, BatAlgorithm, ParticleSwarmOptimization
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score, accuracy_score
from imblearn.under_sampling import RandomUnderSampler

from preprocessing import get_train_test_data
from benchmark import ClassificationBenchmark
from optimizer import optimize
import warnings

warnings.filterwarnings("ignore")

### Loading and preparing training and testing data

In [14]:
%%time

X_train, X_test, y_train, y_test = get_train_test_data('train_transaction.csv', 'train_identity.csv')

all_columns:
cat_features:
num_features:
CPU times: user 2min 33s, sys: 2min 34s, total: 5min 8s
Wall time: 5min 33s


In [15]:
X_train, y_train = RandomUnderSampler().fit_resample(X_train, y_train)

### Optimizing columns using Firefly algorithm and Decision Tree Classifier

In [19]:
benchmark = ClassificationBenchmark(DecisionTreeClassifier, accuracy_score, X_train, y_train, X_test, y_test)

In [20]:
%%time

columns = optimize(benchmark, FireflyAlgorithm(), 100)

 20%|██        | 1/5 [28:18<1:53:13, 1698.31s/it]

--------------
Run 1
--------------
Score: 0.8095471940935415
Number of features selected: 115





 20%|██        | 1/5 [31:25<2:05:43, 1885.94s/it]


KeyboardInterrupt: 